In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Piauí - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego,Piauí - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.649218,24.669760,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,8.192266,21.844
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.649661,25.167145,2.139102e+07,1.359835e+06,6.065065,1.869506e+07,8.186156,15.814
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.650104,25.520404,2.142142e+07,1.361626e+06,6.068988,1.870864e+07,8.180046,16.067
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.650548,25.519725,2.145181e+07,1.363417e+06,6.072912,1.872222e+07,8.173937,14.938
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.650991,25.929312,2.148221e+07,1.365208e+06,6.076835,1.873580e+07,8.167827,17.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.957215,NaN,NaN,NaN,NaN,NaN,77.776
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.965575,NaN,NaN,NaN,NaN,NaN,72.377
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.977734,NaN,NaN,NaN,NaN,NaN,71.791
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.994123,NaN,NaN,NaN,NaN,NaN,71.015


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.231191,0.000000,0.000000,0.000000,0.000000,0.078897
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.010908,0.245044,0.005814,0.017769,0.008894,0.008766,0.077028
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.021817,0.254883,0.011628,0.035538,0.017788,0.017531,0.075160
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.032725,0.254864,0.017443,0.053307,0.026682,0.026297,0.073291
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.043633,0.266272,0.023257,0.071077,0.035577,0.035063,0.071422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.978318,0.028551,0.973453,0.793501,0.961232,0.960092,0.945259
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.976544,0.020943,0.970065,0.797449,0.956913,0.955740,0.944716
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.974770,0.013641,0.966677,0.801396,0.952594,0.951387,0.944172
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.972996,0.006656,0.963289,0.805344,0.948275,0.947035,0.943629


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Piauí - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.231191,0.000000,0.000000,0.000000,0.000000,0.078897
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.010908,0.245044,0.005814,0.017769,0.008894,0.008766,0.077028
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.021817,0.254883,0.011628,0.035538,0.017788,0.017531,0.075160
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.032725,0.254864,0.017443,0.053307,0.026682,0.026297,0.073291
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.043633,0.266272,0.023257,0.071077,0.035577,0.035063,0.071422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.990867,0.532244,0.976314,0.748250,0.974121,0.975577,0.999216
158,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.991698,0.510976,0.978467,0.748313,0.976474,0.977798,0.998431
159,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.992528,0.489491,0.980620,0.748377,0.978826,0.980018,0.997647
160,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.993358,0.467789,0.982774,0.748441,0.981179,0.982238,0.996863


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
157    54.656
158    69.984
159    58.643
160    72.337
161    70.968
Name: Piauí - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
126,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.720785,0.911121,0.839906,0.986255,0.918684,0.911178,0.690493
127,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.720704,0.897071,0.847037,0.988546,0.924256,0.916950,0.706656
128,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.720624,0.883315,0.854168,0.990837,0.929828,0.922721,0.722819
129,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.720544,0.870408,0.861300,0.993128,0.935400,0.928493,0.738982
130,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.720464,0.858682,0.868431,0.995418,0.940972,0.934265,0.755145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.978318,0.028551,0.973453,0.793501,0.961232,0.960092,0.945259
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.976544,0.020943,0.970065,0.797449,0.956913,0.955740,0.944716
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.974770,0.013641,0.966677,0.801396,0.952594,0.951387,0.944172
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.972996,0.006656,0.963289,0.805344,0.948275,0.947035,0.943629


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
#                         validation_split=0.15,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1470039473, 3733917898, 385224538, 3005128575, 1281400932, 4154338123, 4187196227, 2485305851, 3140281352, 685414895]


Step: 0 ___________________________________________
val_loss: 308.0115661621094
winner_seed: 1470039473


Step: 1 ___________________________________________
val_loss: 308.36773681640625


Step: 2 ___________________________________________
val_loss: 16.35261344909668
winner_seed: 385224538


Step: 3 ___________________________________________
val_loss: 22.326622009277344


Step: 4 ___________________________________________
val_loss: 308.163330078125


Step: 5 ___________________________________________
val_loss: 307.9754638671875


Step: 6 ___________________________________________
val_loss: 18.61030387878418


Step: 7 ___________________________________________
val_loss: 307.95513916015625


Step: 8 ___________________________________________
val_loss: 17.539947509765625


Step: 9 ___________________________________________
val_loss: 308.0856018066406


final_see

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=3430781553)

Epoch 1/10000
4/4 [==============================] - 3s 134ms/step - loss: 3589.0229 - val_loss: 3776.4639
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 3445.9500 - val_loss: 3652.6238
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 3348.6621 - val_loss: 3558.7646
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 3264.8469 - val_loss: 3481.8643
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 3193.4568 - val_loss: 3419.2334
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 3137.7229 - val_loss: 3366.2451
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 3090.4111 - val_loss: 3322.6084
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 3047.9561 - val_loss: 3288.1509
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 3012.8271 - val_loss: 3261.0857
Epoch 10/10000
4/4 [=============================

Epoch 78/10000
4/4 [==============================] - 0s 10ms/step - loss: 2394.5229 - val_loss: 2643.0959
Epoch 79/10000
4/4 [==============================] - 0s 9ms/step - loss: 2387.6282 - val_loss: 2635.8372
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 2380.9268 - val_loss: 2628.6265
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 2374.1482 - val_loss: 2621.4761
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 2367.3066 - val_loss: 2614.3569
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 2360.3413 - val_loss: 2607.1968
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 2353.6353 - val_loss: 2599.9910
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 2346.8025 - val_loss: 2592.8311
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 2340.2102 - val_loss: 2585.6309
Epoch 87/10000
4/4 [=======================

4/4 [==============================] - 0s 9ms/step - loss: 1925.6680 - val_loss: 2146.5444
Epoch 155/10000
4/4 [==============================] - 0s 9ms/step - loss: 1919.9227 - val_loss: 2140.6748
Epoch 156/10000
4/4 [==============================] - 0s 9ms/step - loss: 1914.5076 - val_loss: 2134.7397
Epoch 157/10000
4/4 [==============================] - 0s 9ms/step - loss: 1909.0044 - val_loss: 2128.8877
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 1903.4581 - val_loss: 2123.0837
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 1897.9803 - val_loss: 2117.2893
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 1892.4833 - val_loss: 2111.5186
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 1887.1443 - val_loss: 2105.6709
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 1881.7029 - val_loss: 2099.8574
Epoch 163/10000
4/4 [===========================

4/4 [==============================] - 0s 9ms/step - loss: 1542.3446 - val_loss: 1737.2908
Epoch 231/10000
4/4 [==============================] - 0s 10ms/step - loss: 1537.7271 - val_loss: 1732.4215
Epoch 232/10000
4/4 [==============================] - 0s 9ms/step - loss: 1533.1938 - val_loss: 1727.6021
Epoch 233/10000
4/4 [==============================] - 0s 10ms/step - loss: 1528.6787 - val_loss: 1722.7808
Epoch 234/10000
4/4 [==============================] - 0s 10ms/step - loss: 1524.1874 - val_loss: 1717.9819
Epoch 235/10000
4/4 [==============================] - 0s 9ms/step - loss: 1519.7323 - val_loss: 1713.1907
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 1515.2975 - val_loss: 1708.4066
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 1510.8073 - val_loss: 1703.6497
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 1506.3976 - val_loss: 1698.8790
Epoch 239/10000
4/4 [=============================

4/4 [==============================] - 0s 12ms/step - loss: 1231.5593 - val_loss: 1402.3818
Epoch 307/10000
4/4 [==============================] - 0s 10ms/step - loss: 1227.9001 - val_loss: 1398.3818
Epoch 308/10000
4/4 [==============================] - 0s 10ms/step - loss: 1224.2427 - val_loss: 1394.4205
Epoch 309/10000
4/4 [==============================] - 0s 9ms/step - loss: 1220.6643 - val_loss: 1390.4918
Epoch 310/10000
4/4 [==============================] - 0s 9ms/step - loss: 1216.9225 - val_loss: 1386.6464
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 1213.4020 - val_loss: 1382.6899
Epoch 312/10000
4/4 [==============================] - 0s 10ms/step - loss: 1209.7489 - val_loss: 1378.7311
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 1206.1079 - val_loss: 1374.7876
Epoch 314/10000
4/4 [==============================] - 0s 10ms/step - loss: 1202.5536 - val_loss: 1370.8208
Epoch 315/10000
4/4 [=========================

...

4/4 [==============================] - 0s 10ms/step - loss: 296.5286 - val_loss: 309.1147
Epoch 1923/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5528 - val_loss: 309.0877
Epoch 1924/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5713 - val_loss: 309.1649
Epoch 1925/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5285 - val_loss: 309.1676
Epoch 1926/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5294 - val_loss: 309.1546
Epoch 1927/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5284 - val_loss: 309.1447
Epoch 1928/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5328 - val_loss: 309.1616
Epoch 1929/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5281 - val_loss: 309.1526
Epoch 1930/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5278 - val_loss: 309.1475
Epoch 1931/10000
4/4 [==============================] - 0

4/4 [==============================] - 0s 8ms/step - loss: 296.5432 - val_loss: 309.2259
Epoch 2000/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5325 - val_loss: 309.2098
Epoch 2001/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5421 - val_loss: 309.2201
Epoch 2002/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5450 - val_loss: 309.1917
Epoch 2003/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5287 - val_loss: 309.1569
Epoch 2004/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5215 - val_loss: 309.1143
Epoch 2005/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5301 - val_loss: 309.0508
Epoch 2006/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5254 - val_loss: 309.0264
Epoch 2007/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5267 - val_loss: 309.0117
Epoch 2008/10000
4/4 [==============================] - 0s 

4/4 [==============================] - 0s 10ms/step - loss: 296.5249 - val_loss: 309.0039
Epoch 2077/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5287 - val_loss: 309.0658
Epoch 2078/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5300 - val_loss: 309.0751
Epoch 2079/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5316 - val_loss: 309.0717
Epoch 2080/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5291 - val_loss: 309.0772
Epoch 2081/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5295 - val_loss: 309.0467
Epoch 2082/10000
4/4 [==============================] - 0s 11ms/step - loss: 296.5248 - val_loss: 309.0051
Epoch 2083/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5265 - val_loss: 308.9807
Epoch 2084/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5265 - val_loss: 308.9543
Epoch 2085/10000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 8ms/step - loss: 296.5293 - val_loss: 308.8286
Epoch 2154/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5342 - val_loss: 308.8468
Epoch 2155/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5296 - val_loss: 308.8583
Epoch 2156/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5284 - val_loss: 308.8358
Epoch 2157/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5258 - val_loss: 308.7747
Epoch 2158/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5199 - val_loss: 308.7323
Epoch 2159/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5362 - val_loss: 308.6789
Epoch 2160/10000
4/4 [==============================] - 0s 8ms/step - loss: 296.5270 - val_loss: 308.6566
Epoch 2161/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5296 - val_loss: 308.6287
Epoch 2162/10000
4/4 [==============================] - 0s 8ms/

4/4 [==============================] - 0s 9ms/step - loss: 296.5469 - val_loss: 309.7501
Epoch 2231/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5710 - val_loss: 309.6981
Epoch 2232/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5564 - val_loss: 309.7128
Epoch 2233/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5661 - val_loss: 309.6489
Epoch 2234/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5472 - val_loss: 309.6227
Epoch 2235/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5421 - val_loss: 309.6288
Epoch 2236/10000
4/4 [==============================] - 0s 10ms/step - loss: 296.5488 - val_loss: 309.6632
Epoch 2237/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5472 - val_loss: 309.6682
Epoch 2238/10000
4/4 [==============================] - 0s 9ms/step - loss: 296.5468 - val_loss: 309.6473
Epoch 2239/10000
4/4 [==============================] - 0s 9

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,57.945511,57.945511,57.945511,57.945503,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945511,57.945503,57.945511,57.945511,57.945503,57.945511,57.945511,57.945507,57.945507,57.945511,57.945511,57.945507,57.945511,57.945507
Target,75.913,74.774,72.484,71.534,70.32,67.364,59.688,40.259,41.239,45.035,42.477,57.619,53.109,63.141,55.58,60.001,62.767,45.308,57.142,46.283,43.39,40.824,55.063,56.474,68.133,68.642,64.741,71.269,67.631,62.845
Error,17.967491,16.828491,14.53849,13.588493,12.374489,9.418488,1.742489,17.686512,16.706512,12.910511,15.46851,0.326511,4.83651,5.195488,2.365509,2.055489,4.821487,12.637505,0.803513,11.66251,14.555504,17.12151,2.882511,1.471508,10.187496,10.696487,6.795486,13.32349,9.685486,4.899494


In [38]:
display(mae)
display(mape)

9.518466

0.17406164

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[695.346]] - Target[718.706]| =  Error: [[23.359985]]; MAPE:[[0.03250284]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[695.346]] - Target[639.082]| =  Error: [[56.264038]]; MAPE:[[0.08803885]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[347.67303]] - Target[403.26099999999997]| =  Error: [[55.58795]]; MAPE:[[0.1378461]]


[array([[23.359985]], dtype=float32),
 array([[56.264038]], dtype=float32),
 array([[55.58795]], dtype=float32)]

45.07066

0.086129256